# Mutual Information Matrices from Attention Graphs

By computing the inverse mutual information distance matrix for each piece of text, we can compare the semantic similarity between the two pieces of text by comparing their persistent homology and computing the bottleneck distance between their persistence diagrams or barcodes. Here we define 

$$ MI^{-1}(x_i, x_j) = 2\ln 2 - MI(x_i, x_j)$$

where $MI(x_i, x_j)$ is the mutual information. By computing the inverse mutual information distance matrix for each piece of text, we can use persistent homology to compare the semantic similarity between the two pieces of text. Persistent homology is a mathematical tool that is used to study the topological features of data across multiple scales.

In this context, we can use persistent homology to study the topological features of the distance matrix that is computed from the inverse mutual information between tokens in the two pieces of text. The persistent homology of a distance matrix is captured by its persistence diagram or barcode, which is a graphical representation of the topological features of the matrix across multiple scales.

We can compute the persistent homology barcodes for each distance matrix using the Rips complex, which is a standard way to construct a simplicial complex from a distance matrix. We can then convert the persistence diagrams to NumPy arrays and compute the bottleneck distance between them. The bottleneck distance between two persistence diagrams is a measure of their similarity and captures the distance between the topological features of the two matrices.

By comparing the bottleneck distance between the persistence diagrams of the two matrices, we can obtain a quantitative measure of the semantic similarity between the two pieces of text. If the bottleneck distance is small, it indicates that the two matrices have similar topological features and hence the two pieces of text are semantically similar. Conversely, if the bottleneck distance is large, it indicates that the two matrices have different topological features and hence the two pieces of text are semantically dissimilar.

Therefore, by using persistent homology and the bottleneck distance, we can compare the semantic similarity between two pieces of text that are represented as distance matrices based on the inverse mutual information between tokens. Here, we compute a distance matrix based on the mutual information between tokens in a text corpus using the attention scores. Once this is computed for two pieces of text we wish to compare topologically, we can run the following code to compute the persistence barcodes and the bottleneck distance between them. 

In [ ]:
pip install transformers -q

In [ ]:
import numpy as np
import gudhi
import matplotlib.pyplot as plt

def compute_bottleneck_distance(n):
    # Generate two random distance matrices of size n
    D1 = np.random.rand(n, n)
    D2 = np.random.rand(n, n)

    # Compute the persistent homology barcodes for each matrix
    rips1 = gudhi.RipsComplex(distance_matrix=D1)
    st1 = rips1.create_simplex_tree(max_dimension=2)
    diag1 = st1.persistence()
    rips2 = gudhi.RipsComplex(distance_matrix=D2)
    st2 = rips2.create_simplex_tree(max_dimension=2)
    diag2 = st2.persistence()

    # Plot the barcodes
    gudhi.plot_persistence_barcode(diag1)
    plt.title("Barcode for Distance Matrix 1")
    plt.show()
    gudhi.plot_persistence_barcode(diag2)
    plt.title("Barcode for Distance Matrix 2")
    plt.show()

    # Convert the persistence diagrams to NumPy arrays
    diag1 = np.array([[float(birth), float(death)] for _, (birth, death) in diag1])
    diag2 = np.array([[float(birth), float(death)] for _, (birth, death) in diag2])

    # Compute the bottleneck distance between the barcodes
    return gudhi.bottleneck_distance(diag1, diag2)


This first uses all layers and heads to compute a single mutual information matrix. 

In [ ]:
import torch
import numpy as np
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
from transformers import GPT2Tokenizer, GPT2Model

# Set up the GPT-2 model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2Model.from_pretrained(model_name, output_attentions=True)

def mutual_information(p_ij, p_i, p_j):
    if p_i == 0 or p_j == 0 or p_ij == 0:
        return 0
    pmi = np.log2(p_ij / (p_i * p_j))
    mi = np.maximum(0, pmi)
    return mi

def compute_mutual_information(text):
    # Tokenize the input text and convert it to a tensor
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)

    # Get the model output with attention weights
    outputs = model(input_ids)

    # Initialize the mutual information matrix
    sequence_length = input_ids.shape[1]
    mi_matrix = np.zeros((sequence_length, sequence_length))

    # Iterate through each layer of the output and extract the attention weights
    for layer, attention in enumerate(outputs.attentions):
        weights = attention[0].detach().numpy()  # Shape: (num_heads, sequence_length, sequence_length)
        for head in range(weights.shape[0]):
            # Compute the probabilities of each token
            token_probs = defaultdict(float)
            for i in range(sequence_length):
                token_probs[input_ids[0][i].item()] += 1
            token_probs = {k: v / sequence_length for k, v in token_probs.items()}

            # Compute the pairwise mutual information between tokens
            for i in range(sequence_length):
                for j in range(sequence_length):
                    if i == j:
                        continue
                    p_ij = weights[head][i][j]
                    p_i = token_probs[input_ids[0][i].item()]
                    p_j = token_probs[input_ids[0][j].item()]
                    mi = mutual_information(p_ij, p_i, p_j)
                    mi_matrix[i][j] += mi

    return mi_matrix


In [ ]:
compute_mutual_information("The quick brown fox jumped over the lazy dog")

The next two compute a mutual information matrix for each layer and head.  

In [ ]:
import numpy as np
from collections import defaultdict

model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

def compute_mutual_information(model, tokenizer, text):
    # Tokenize the input text and convert it to a tensor
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)

    # Get the model output with attention weights
    outputs = model(input_ids, output_attentions=True)

    # Define a function to compute the probability of each token
    def compute_probabilities(weights):
        # Compute the marginal probability of each token
        p_i = np.sum(weights, axis=1) / np.sum(weights)
        # Compute the joint probability of each pair of tokens
        p_ij = weights / np.sum(weights)
        return p_i, p_ij

    # Initialize a dictionary to store the mutual information matrices for each layer and head
    mi_matrices = defaultdict(list)

    # Iterate through each layer of the output and extract the attention weights
    for layer, attention in enumerate(outputs.attentions):
        # Convert the attention weights to a weighted adjacency matrix
        weights = attention[0].detach().numpy()  # Shape: (num_heads, sequence_length, sequence_length)
        # Iterate through each head of the layer
        for head in range(weights.shape[0]):
            # Compute the probability of each token
            p_i, p_ij = compute_probabilities(weights[head])
            # Compute the pointwise mutual information between each pair of tokens
            pmi = np.log2(p_ij / np.outer(p_i, p_i))
            pmi[np.isnan(pmi)] = 0.0  # Set NaN values to 0.0
            mi = np.maximum(0, pmi)  # Set negative values to 0.0
            # Store the mutual information matrix for this layer and head
            mi_matrices[(layer, head)].append(mi)

    return mi_matrices


In [ ]:
text = "The quick brown fox jumps over the lazy dog."
compute_mutual_information(model, tokenizer, text)

In [ ]:
import numpy as np
from scipy.special import xlogy

def compute_mutual_information(input_text, tokenizer, model):
    # Tokenize the input text and convert it to a tensor
    input_ids = torch.tensor(tokenizer.encode(input_text)).unsqueeze(0)

    # Get the model output with attention weights
    outputs = model(input_ids)

    # Compute the pairwise mutual information matrix for each layer and head
    mi_matrices = []
    for layer, attention in enumerate(outputs.attentions):
        weights = attention[0].detach().numpy()  # Shape: (num_heads, sequence_length, sequence_length)
        mi_matrix = np.zeros((weights.shape[0], weights.shape[1], weights.shape[2]))
        for head in range(weights.shape[0]):
            for i in range(weights.shape[1]):
                for j in range(weights.shape[2]):
                    if i == j:
                        continue
                    p_ij = weights[head][i][j]
                    p_i = weights[head][i].sum()
                    p_j = weights[head][:,j].sum()
                    eps = 1e-9
                    pmi = xlogy(p_ij + eps, p_ij + eps / (p_i * p_j + eps))
                    mi_matrix[head][i][j] = pmi
        mi_matrices.append(mi_matrix)
    return mi_matrices


In [ ]:
compute_mutual_information(text, tokenizer, model)